In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
## Load the trained model, scaler pickel file and one hot encoder pickel file
model = load_model('churn_model.h5')

# load the encoder and scaler
with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_geo = pickle.load(f)

with open('onehot_encoder_geo.pkl', 'rb') as f:
    onehot_encoder = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [4]:
# Example new customer data for prediction
new_customer = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
geo_encoded = onehot_encoder.transform([[new_customer['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder.get_feature_names_out(['Geography']))

c:\Users\FPK1COB\Documents\Learning\ANN-Projects\Churn\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [ ]:
new_customer_df = pd.DataFrame([new_customer])
new_customer_df = pd.concat([new_customer_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
new_customer_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
## Encode Categorical values
new_customer_df['Gender'] = label_encoder_geo.transform([new_customer['Gender']])
new_customer_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
new_customer_df = scaler.transform(new_customer_df)
new_customer_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [10]:
## Prediction
prediction = model.predict(new_customer_df)
prediction

1/1 [==============================] - 0s 81ms/step


array([[0.03752564]], dtype=float32)

In [11]:
prediction_probability = prediction[0][0]

In [12]:
prediction_probability

0.03752564

In [13]:
if prediction_probability > 0.5:
    print(f"The customer is likely to churn with a probability of {prediction_probability:.2f}")
else:
    print(f"The customer is unlikely to churn with a probability of {prediction_probability:.2f}")

The customer is unlikely to churn with a probability of 0.04
